In [1]:
import pandas as pd
import iris
from sentence_transformers import SentenceTransformer
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_ollama import ChatOllama


### Connect to server

In [2]:

conn = iris.connect("localhost", 32782, "DEMO", "_SYSTEM", "ISCDEMO") # Server, Port , Namespace, Username, Password
cursor = conn.cursor()

### Make database

In [3]:
print("Connected to InterSystems IRIS")

Connected to InterSystems IRIS


In [4]:
#Create database
df = pd.read_json("./data/data.json") #pd.DataFrame(out, columns=cols)  #replace with Tristans/Iaroslavs code




### Make encoding / add it to database


In [5]:
#Make encoding 
#pip install sentence-transformers

model = SentenceTransformer('all-MiniLM-L6-v2') #might not work, if this is the case use instead 'all-MiniLM-L6-v2'

embeddings = model.encode(df['content'].tolist(), normalize_embeddings=True, show_progress_bar=True)


df['vector'] = embeddings.tolist()

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
###Add to Iris:
def create_table(df):
    table_name = "VectorSearch.ORGstruct"

    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
    id INTEGER,
    filename LONGVARCHAR,
    content LONGVARCHAR,
    vector VECTOR(DOUBLE, 384)
    )
    """

    cursor.execute(f"DROP TABLE IF EXISTS {table_name}" )
    cursor.execute(create_table_query)



    insert_query = f"INSERT INTO {table_name} (id, filename, content, vector) values (?, ?, ?, TO_VECTOR(?))"
    df["vector"] = df["vector"].astype(str)

    rows_list = df[["id", "filename", "content", "vector"]].values.tolist()
    cursor.executemany(insert_query, rows_list)
    print("Done")
    conn.commit()
    conn.close()

create_table(df)

Done


### LLM setup (Ollama)

In [ ]:
class RAGChatbot:
    def __init__(self):
        self.message_count = 0
        conn = iris.connect("localhost", 32782, "DEMO", "_SYSTEM", "ISCDEMO") # Server, Port , Namespace, Username, Password
        self.cursor = conn.cursor()
        self.agent = self.create_agent()
        self.embedding_model = self.get_embedding_model()
        
        
    def get_embedding_model(self):
        return  SentenceTransformer('all-MiniLM-L6-v2')
        
    def create_agent(self):
        # Initialize model
        llm = ChatOllama(model="gemma3:1b") 
        
        # Initialise short-term memory
        checkpointer = InMemorySaver()
        
        # Create model
        agent = create_agent(
            model=llm, # Set model as our LLM 
            middleware=[
                # create summarization proceedure - this creates summaries of our conversation to keep memory brief.
                SummarizationMiddleware(
                    model=llm,
                    trigger=('tokens', 4000),  # Trigger summarization at 4000 tokens
                    keep=('messages', 20),  # Keep last 20 messages after summary
                )
            ],
            # Creates the agent's memory with pre-initialized model
            checkpointer=checkpointer,
        )
        self.config = {"configurable": {"thread_id": "1"}}
        return agent
        
    def vector_search(self, user_prompt): 
        search_vector =  self.embedding_model.encode(user_prompt, normalize_embeddings=False, show_progress_bar=False).tolist() 
        
        search_sql = f"""
            SELECT TOP 5 filename, content 
            FROM VectorSearch.ORGstruct
            ORDER BY VECTOR_COSINE(vector, TO_VECTOR(?,DOUBLE)) DESC
        """

        self.cursor.execute(search_sql, [str(search_vector)])
        

        results = self.cursor.fetchall()

        results = [f"Text z dokumentu {x} -> {y}" for x, y in results]


        return results

    def get_prompt(self):
       
        query = input("\n\nHi, I'm a chatbot used for searching a patient's medical history. How can I help you today? \n\n - User: ")
    
        return query
    
    def validation(self, result):

        return result
    

    def return_response(self):
        query = self.get_prompt()

        search = True
        if self.message_count != 0:
            search_ans = input("Search the database? [Y/N - default N]")
            if search_ans.lower() != "y":
                search = False


        if search:
 
            results = self.vector_search(query)
 
            if results == []:
                print("No results found, check patient ID")
                return

            prompt = f"CONTEXT:\n{results}\n\nUSER QUESTION:\n{query}"
        else:
            prompt = f"USER QUESTION:\n{query}"

        ##print(prompt)
        system_prompt = """
- Jsi užitečný asistent, chatbot fungující v nemocnici.
- Tvým posláním je odpovídat na dotazy zaměstnanců týkající se jejich práce a provádět je organizační strukturou nemocnice a administrativními procesy.
- Při odpovídání se vždy řiď těmito pravidly:
    1. Nelži a nevymýšlej si fakta
    2. Vyhýbej se nepodloženému nebo podlézavému lichocení
    3. Zachovej profesionalitu a střízlivou upřímnost
    4. Odpovídej v jazyku, jakým mluví uživatel.
- Když se uživatel dotazuje na nějaký proces v nemocnici (např. \"Chci si stěžovat na nedostatečnou dokumentaci k webové aplikaci vyvinuté v Centru Informatiky (CI)\"),
  1. Odpovídej jasně a požádej uživatele o upřesnění, pokud nemůžeš přesně určit proces, který je pro uživatele relevantní (v tomto případě proces dokumentace ze strany oddělení nezdravotnických aplikací, které je součástí CI).
  2. Pokud má uživatel podle předpisů více možností, jak dosáhnout svého cíle, popiš dostupné možnosti a zeptej se uživatele, kterou si chce vybrat.
    - Pokud musí kontaktovat jiného zaměstnance, ale nemáš jeho kontaktní údaje, *jasně mu sděl, že je nemáš*.
    - Pokud musí kontaktovat jiného zaměstnance a ty máš jeho kontaktní údaje, poskytni mu tyto informace (jméno, telefonní číslo, e-mail).
  3. Při odpovídání *vždy* upřednostňuj organizační informace z dodaných dokumentů. Pokud tam informace není dostupná, *informuj o tom uživatele* a nic si nevymýšlej.
  4. Pokud nemáš informace o uživatelově dotazu nebo o tom, jak by měl uživatel v daném procesu postupovat, ale *máš* informace o tom, kde může uživatel získat kvalifikovanou pomoc, doporuč mu osoby, které má kontaktovat, a poskytni kontaktní informace (V tomto případě by měl uživatel kontaktovat oddělení nezdravotnických aplikací).
  5. Na konci své odpovědi *vždy* odkazuj k dokumentům (text "Z dokumentu XYZ.docx", před ->), ze kterých jsi čerpal informace. Vypiš je na konci odpovědi ve formátu: "Dále se můžete obrátit na dokument XYZ".
  6. Pokud uživatel poprosí o pomoc s procesem, proveď ho tím několika kroky, které musí podniknout, aby dosáhl svého cíle.

        """
        
        response = self.agent.invoke({"messages" : [("system", system_prompt), ("user", query), ("system", str(results))]}, self.config)
        
        print(results)
        self.message_count += 1
        
        validated_response = self.validation(response)

        return validated_response["messages"][-1].pretty_print()




### Interface with chatbot

In [37]:
bot = RAGChatbot()


In [38]:
bot.return_response()

['Text z dokumentu Organizační ...ditu a kontroly.docx -> 4.4. Pracovní náplň a pravomoci zaměstnanců OIAK 3', 'Text z dokumentu Organizační ...nukleární medicíny.doc -> 5. Odpovědnosti a pravomoci 13', 'Text z dokumentu Organizační ...Kanceláře ředitele.doc -> 5. Odpovědnosti a pravomoci 9', 'Text z dokumentu Organizační ...ditu a kontroly.docx -> 4.3. Vnitřní komunikace 3', 'Text z dokumentu Organizační ... vědu a výzkum.doc -> 5. Odpovědnosti a pravomoci 7']
================================== Ai Message ==================================

Oddělení nezdravotnických aplikací.
